## STAGE 9: Annotation File Creation
Follow these steps:

1. Update the variables: `COMMENT_FILES_FOLDER` and `POST_FILE`


In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

DATE_RANGE = '20200427_20200430'
isGroup = False

if isGroup:
    COMMENT_FILES_FOLDER = '../data/exportcomments-outputs/groups_{}'.format(DATE_RANGE)
    POST_FILE = '../data/crowdtangle-groups/processed_{}.csv'.format(DATE_RANGE)
else:
    COMMENT_FILES_FOLDER = '../data/exportcomments-outputs/{}'.format(DATE_RANGE)
    POST_FILE = '../data/crowdtangle-posts/processed_{}.csv'.format(DATE_RANGE)
URL_PREFIX = 'https://www.facebook.com/profile.php?id='
print(COMMENT_FILES_FOLDER)

../data/exportcomments-outputs/20200427_20200430


In [2]:
all_files =  glob(COMMENT_FILES_FOLDER+'/processed/comments_*.csv')
print(len(all_files))

382


## Add Post URLs

In [3]:
postdf = pd.read_csv(POST_FILE, sep='~')
print(postdf.shape)
display(postdf.head(2))

(400, 34)


,Page Name,User Name,Facebook Id,Likes at Posting,Created,Type,Likes,Comments,Shares,Love,...,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Overperforming Score,commentsFile,PostId,MessageUni,MsgUniCleanSeg
0,Myanmarthandawsint Newspaper,myanmarherald,240972982664335,297271,2020-04-29 18:50:35 MMT,Photo,2899,716,5386,29,...,NaN,Photos from Myanmarthandawsint Newspaper's post,NaN,NaN,NaN,208.85,../data/exportcomments-outputs/20200427_202004...,20200427_0000,ဗွီဒီယို Call မကိုင်တတ်သောသူခိုးအား ပုဒ်မ ၁၄၄ ...,ဗွီ ဒီ ယို Call မ ကိုင် တတ် သော သူခိုး အား ပုဒ...
1,မန္တလေး True News,mandalaytruenews,110968113662369,36057,2020-04-28 21:19:29 MMT,Status,3439,436,2479,9,...,NaN,NaN,NaN,NaN,NaN,94.62,../data/exportcomments-outputs/20200427_202004...,20200427_0001,မန္တလေး၊ ဧပြီ ၂၈ မန္တလေး၊အောင်မြေသာစံမြို့နယ်၊...,မန္တလေး ၊ ဧပြီ ၂၈ မန္တလေး ၊ အောင်မြေ သာ စံ မြိ...


- commentsFile name is matched from the postdf and the post url is fetched.

In [4]:
for f in tqdm(all_files):
    commentFileId = f.split('/')[-1].split('.')[0]
    commentFile = '{}/{}.xlsx'.format(COMMENT_FILES_FOLDER, commentFileId)
    tmp = pd.read_csv(f, sep='~')
    tmp['PostURL'] = postdf.loc[postdf.commentsFile==commentFile,'URL'].values[0]
    tmp.to_csv(f, index=False, sep='~')

100%|██████████| 382/382 [00:08<00:00, 46.92it/s]


## Merge
- Merge the comments files now that we have all the columns we need

In [5]:
df_from_each_file = (pd.read_csv(f, sep='~') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv(COMMENT_FILES_FOLDER+'/processed/merged.csv', index=False)

### Columns selection for annotation file

In [6]:
COMMENT_FILES_FOLDER+'/processed/merged.csv'

'../data/exportcomments-outputs/20200427_20200430/processed/merged.csv'

In [7]:
df = pd.read_csv(COMMENT_FILES_FOLDER+'/processed/merged.csv')
# df['atleast1MM'] = df.atleast1MM.bool()
print(df.shape)
df = df.loc[df.atleast1MM]
df = df.loc[df.char_pass]
print(df.shape)
print(df.columns)

(83845, 15)
(60714, 15)
Index(['cId', 'nId', 'Name (click to view profile)', 'Profile ID', 'Date',
       'Likes', 'Comment', '(view source)', 'postId', 'char_pass', 'MsgUni',
       'atleast1MM', 'MsgUniSeg', 'LexFound', 'PostURL'],
      dtype='object')


## Finding COVID keywords (This step should be somewhere else tbh)

In [8]:
with open('../data/covid.csv') as f:
    covlex = list(set([''.join(l.strip().split()).lower() for l in f]))
print(len(covlex))

27


In [9]:
x = ['~'.join(set([c for c in covlex if c in m.lower()])) for m in tqdm(df.MsgUniSeg)]

100%|██████████| 60714/60714 [00:03<00:00, 16653.74it/s]


In [10]:
df['CovFound'] = [l if len(l)>0 else np.NaN for l in x]
print(sum(~df.CovFound.isna()))
df.head(2)

2382


,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,char_pass,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,CovFound
0,1,0,Aung Than Zaw,ID: 100001267573369,2020-04-27 12:51:43,1,ဗမာလူမျိုးများအပေါ်မွန်တွေသာ သမိုၚ်းဆိုးတစ်ခုရ...,view comment,20200427_0282,True,ဗမာလူမျိုးများအပေါ်မွန်တွေသာ သမိုၚ်းဆိုးတစ်ခုရ...,True,ဗမာ လူမျိုး များ အပေါ် မွန် တွေ သာ သ မိုၚ်း ဆိ...,ဗမာတွေ,https://www.facebook.com/Monstatenews18/posts/...,NaN
1,2,0,Oo Paw,ID: 100008740395328,2020-04-27 13:23:34,0,ပေရာမန္လိက္ဗဿာတီစ၀ါတ္မံင္ရအၥာမန္တံ,view comment,20200427_0282,True,ပရောမန်လိက်ဗဿာတီစဝါတ်မံင်ရအ္စာမန်တံ,True,ပ ရော မန် လိက် ဗ ဿာ တီ စ ဝါတ် မံင် ရအ္စာ မန် တံ,NaN,https://www.facebook.com/Monstatenews18/posts/...,NaN


In [11]:
dfwcov = df.loc[~df.CovFound.isna(),:]
print(dfwcov.shape)

(2382, 16)


## Select Priority FB Pages

In [12]:
with open('../data/fbpage_priority.csv') as f:
    fbpages = tuple([l.strip() for l in f])
print(len(fbpages))

5


In [42]:
mask = [str(url).startswith(fbpages) for url in tqdm(df.PostURL)]

100%|██████████| 38725/38725 [00:00<00:00, 912348.11it/s]


In [43]:
print(sum(mask))
dfwpg = df.loc[mask, :]
dfwpg.head(2)

267


,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,char_pass,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,CovFound
44854,1,0,Si Thu Aung,ID: 100008337798030,2020-05-05 07:23:33,33,ပြည်သူတွေ ငတ်သေနေပြီ အစိုးရဘာလုပ်နေလဲ ကားဂိတ်တ...,view comment,20200430_0155,True,ပြည်သူတွေ ငတ်သေနေပြီ အစိုးရဘာလုပ်နေလဲ ကားဂိတ်တ...,True,ပြည် သူ တွေ ငတ် သေ နေ ပြီ အစိုးရ ဘာ လုပ် နေ လဲ...,NaN,https://www.facebook.com/VoiceofMyanmarNews/po...,NaN
44855,1,1,Sithu Aung,ID: 100008619974774,2020-05-05 08:14:10,18,Si Thu Aung အကုန်ကိုဗစ်ကူးတော့ မင်းတာဝန်ယူမလား?,view comment,20200430_0155,True,Si Thu Aung အကုန်ကိုဗစ်ကူးတော့ မင်းတာဝန်ယူမလား?,True,SiThuAung အကုန် ကို ဗစ် ကူး တော့ မင်း တာဝန် ယူ...,NaN,https://www.facebook.com/VoiceofMyanmarNews/po...,NaN


## Select only HS lex

In [44]:
dfwlex = df.loc[~df.LexFound.isna(),:]
del df

In [45]:
dfwlex.shape

(965, 16)

### Let's use only the lexicon from last 2 dates, since the rest had been used multiple times.

In [46]:
# import sys
# sys.path.append('/home/bupi/Documents/pdy/hs/hsle/src/HsleCandidateGenerationUtils.py')
import HsleCandidateGenerationUtils as hsle
from importlib import reload  
reload(hsle)

lexset = hsle.LoadLexiconSet(take_last_ndates=False, n=None, has_downloaded=True)
print(len(lexset))
dfwlex = dfwlex.loc[[any(l in lf for l in lexset) for lf in dfwlex.LexFound],:]
print(dfwlex.shape)
dfwlex.head()

111
(965, 16)


,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,char_pass,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,CovFound
104,17,1,Lay Swe,ID: 100009571638523,2020-05-03 01:53:30,0,U Htun Htun Win ဖွတ်တေတည်းက တိုင်းပြည်ကချမ်းသာ...,view comment,20200430_0301,True,U Htun Htun Win ဖွတ်တေတည်းက တိုင်းပြည်ကချမ်းသာ...,True,UHtunHtunWin ဖွတ် တေ တည်း က တိုင်းပြည် က ချမ်း...,ဖွတ်,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN
109,17,6,U Htun Htun Win,ID: 100006949516170,2020-05-03 03:14:34,0,Lay Swe ရွှေနန်းထင်တာ ဘာမှမပြောလိုပါ ကိုယ့်အယူ...,view comment,20200430_0301,True,Lay Swe ရွှေနန်းထင်တာ ဘာမှမပြောလိုပါ ကိုယ့်အယူ...,True,LaySwe ရွှေ နန်း ထင် တာ ဘာ မှ မ ပြော လို ပါ ကိ...,သူခိုး~ဖွတ်,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN
117,24,0,Myat Ko Ko,ID: 100000477604435,2020-05-03 01:21:56,0,ဗမာေတြက ေၾကာတယ္ေျပာတာေပါ့ ဒါေၾကာင့္တိုင္းရင္းသ...,view comment,20200430_0301,True,ဗမာတွေက ကြောတယ်ပြောတာပေါ့ ဒါကြောင့်တိုင်းရင်းသ...,True,ဗမာတွေ က ကြော တယ် ပြော တာ ပေါ့ ဒါ ကြောင့် တိုင...,ဗမာတွေ,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN
139,38,0,Yin Minhtun,ID: 100011171671188,2020-05-03 01:44:22,1,ဒေါ်စု တစ်နေ့ သေးဘယ်နှစ်ကြိမ်ပေါက် ချီးဘယ်နှစ်...,view comment,20200430_0301,True,ဒေါ်စု တစ်နေ့ သေးဘယ်နှစ်ကြိမ်ပေါက် ချီးဘယ်နှစ်...,True,ဒေါ် စု တစ်နေ့ သေး ဘယ်နှစ် ကြိမ် ပေါက် ချီး ဘယ...,နီပေါ,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN
169,57,2,Soe Thu Myo,ID: 100024693242076,2020-05-03 02:57:07,0,ဖြတ္ေတြကေတာ့ ငိုသံႀကီးနဲ႔ေျပာေနၾကၿပီးသနားလိုက္...,view comment,20200430_0301,True,ဖွတ်တွေကတော့ ငိုသံကြီးနဲ့ပြောနေကြပြီးသနားလိုက်...,True,ဖွတ် တွေ ကတော့ ငို သံကြီး နဲ့ ပြော နေ ကြ ပြီး ...,ဖွတ်,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN


In [47]:
# from collections.abc import Iterable

# dfwlex['CovFound'] = [';'.join(x) if isinstance(x, Iterable) else '' for x in dfwlex.CovFound]
# dfwcov['CovFound'] = [';'.join(x) if isinstance(x, Iterable) else '' for x in dfwcov.CovFound]
# dfwcov['CovFound'] = dfwcov.CovFound.fillna('')
# dfwlex['postId'] = dfwlex.postId.astype(str)
# dfwcov['postId'] = dfwlex.postId.astype(str)
# dfwlex['cId'] = dfwlex.postId.astype(str)
# dfwcov['cId'] = dfwlex.postId.astype(str)
# dfwlex['nId'] = dfwlex.postId.astype(str)
# dfwcov['nId'] = dfwlex.postId.astype(str)

In [48]:
dfj = dfwlex.merge(dfwcov, on=['postId','cId','nId'], how='inner')
dfj.shape

(97, 29)

In [49]:
# fix col names after merging
dfj = dfj[[c for c in dfj.columns if not c.endswith('_y')]]
dfj.columns = [c.rstrip('_x') for c in dfj.columns]
dfj.columns

Index(['cId', 'nId', 'Name (click to view profile)', 'Profile ID', 'Date',
       'Likes', 'Comment', '(view source)', 'postId', 'char_pass', 'MsgUni',
       'atleast1MM', 'MsgUniSeg', 'LexFound', 'PostURL', 'CovFound'],
      dtype='object')

## Get more data from dfwlex since there isn't enough

In [50]:
# remove already selected
# `bigId` is to join
dfwlex['bigId'] = ['{}-{}-{}'.format(p,c,n) for p,c,n in zip(dfwlex.postId,dfwlex.cId,dfwlex.nId)]
dfj['bigId'] = ['{}-{}-{}'.format(p,c,n) for p,c,n in zip(dfj.postId,dfj.cId,dfj.nId)]
dfdiff = dfwlex.loc[~dfwlex.bigId.isin(dfj.bigId),:] # only dfwlex items that are not in dfj
print(dfdiff.shape)
dfdiff.head(2)

(868, 17)


,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,char_pass,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,CovFound,bigId
104,17,1,Lay Swe,ID: 100009571638523,2020-05-03 01:53:30,0,U Htun Htun Win ဖွတ်တေတည်းက တိုင်းပြည်ကချမ်းသာ...,view comment,20200430_0301,True,U Htun Htun Win ဖွတ်တေတည်းက တိုင်းပြည်ကချမ်းသာ...,True,UHtunHtunWin ဖွတ် တေ တည်း က တိုင်းပြည် က ချမ်း...,ဖွတ်,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN,20200430_0301-17-1
109,17,6,U Htun Htun Win,ID: 100006949516170,2020-05-03 03:14:34,0,Lay Swe ရွှေနန်းထင်တာ ဘာမှမပြောလိုပါ ကိုယ့်အယူ...,view comment,20200430_0301,True,Lay Swe ရွှေနန်းထင်တာ ဘာမှမပြောလိုပါ ကိုယ့်အယူ...,True,LaySwe ရွှေ နန်း ထင် တာ ဘာ မှ မ ပြော လို ပါ ကိ...,သူခိုး~ဖွတ်,https://www.facebook.com/MOIWebportalMyanmar/p...,NaN,20200430_0301-17-6


### Sort them by number of lex found

In [51]:
dfdiff['lex_count'] = [l.count('~')+1 for l in dfdiff.LexFound]
dfdiff.sort_values('lex_count', ascending=False, inplace=True)
dfdiff.head(2)

,cId,nId,Name (click to view profile),Profile ID,Date,Likes,Comment,(view source),postId,char_pass,MsgUni,atleast1MM,MsgUniSeg,LexFound,PostURL,CovFound,bigId,lex_count
56044,3,24,ထြန္းလင္း,ID: 100004399951107,2020-05-04 14:54:21,0,Ko Lu Yoe မအေလိုးစစ်ခွေးပီသတယ် မင်းပထွေးခန့်ထာ...,view comment,20200430_0157,True,Ko Lu Yoe မအေလိုးစစ်ခွေးပီသတယ် မင်းပထွေးခန့်ထာ...,True,KoLuYoe မအေ လိုး စစ်ခွေး ပီသ တယ် မင်း ပထွေး ခန...,သူခိုး~စစ်အာဏာရှင်~စစ်ခွေး~ဖွတ်,https://www.facebook.com/NewsWatchJournal/post...,NaN,20200430_0157-3-24,4
9613,246,0,Aung Thu,ID: 100009409537010,2020-05-02 15:50:04,0,မင်းတိုစ်ဖွတ်လုပ်စားခွေးသားများသည်အောက်တန်းစား...,view comment,20200430_0054,True,မင်းတိုစ်ဖွတ်လုပ်စားခွေးသားများသည်အောက်တန်းစား...,True,မင်း တိုစ် ဖွတ် လုပ်စား ခွေးသား များသည် အောက်တ...,သူခိုး~အောက်တန်းစား~ခွေးသား~ဖွတ်,https://www.facebook.com/MizzimaDaily/posts/32...,NaN,20200430_0054-246-0,4


### Now we just have to take top N from `dfdiff`
## Flatten!

In [52]:
DesiredN = 500
dfdiff = dfdiff[dfj.columns]
dfj_ = dfj
dfj = pd.concat([dfj, dfdiff.iloc[:DesiredN-dfj.shape[0],:]])
print(dfj.shape)

(500, 17)


In [53]:
print(dfj.shape)
cols = ['LexFound','postId','cId','nId','MsgUniSeg','Profile ID','PostURL','Date','CovFound']
dfx = dfj.loc[:,cols]
dfx['date'] = [d.split()[0] for d in dfx.Date]
dfx['time'] = [d.split()[1] for d in dfx.Date]
dfx.drop('Date', axis=1, inplace=True)
print(dfx.shape)
dfx.drop_duplicates(['MsgUniSeg'], inplace=True)
dfx['ProfileUrl'] = ['{}{}'.format(URL_PREFIX, i.split()[-1]) for i in dfx['Profile ID']]
print(dfx.shape)
dfx.head(3)

(500, 17)
(500, 10)
(493, 11)


,LexFound,postId,cId,nId,MsgUniSeg,Profile ID,PostURL,CovFound,date,time,ProfileUrl
0,သူခိုး,20200430_0200,11,0,ကျပ် ငွေ 200000( ၂ သိန်း ) ကို ဖုန်း လေး တစ် ခ...,ID: 100043931886875,https://www.facebook.com/yangonregiongovernmen...,covid,2020-05-06,19:03:54,https://www.facebook.com/profile.php?id=100043...
1,မြို့သား,20200430_0055,15,0,ကျွန်မ က မြ ဝ တီ သူမ ဟုတ် ပါ ဘူး ။ ရန် ကုန် က ...,ID: 100045639997751,https://www.facebook.com/1233832270078079/post...,မြို့သား,2020-05-03,04:11:45,https://www.facebook.com/profile.php?id=100045...
2,မြို့သား,20200430_0055,98,0,အဓိက က မြို့ ရဲ့ ဂုဏ်သိက္ခာ နစ်နာ တယ် ရောဂါကပ်...,ID: 100005686590684,https://www.facebook.com/1233832270078079/post...,မြို့သား~ရောဂါ,2020-05-03,07:28:36,https://www.facebook.com/profile.php?id=100005...


In [54]:
dflat = pd.DataFrame(index=dfx.columns)
dfextra = pd.DataFrame(index=dfx.columns)
total = 0
for k in tqdm(range(dfx.shape[0])):
    if '~' not in dfx.iloc[k,0]: # 0 is LexFound
        dflat[k] = dfx.iloc[k,:].values
        total += 1
    else:
        lexes = dfx.iloc[k,0].split('~')
        tmp = dfx.iloc[k,:]
        for lex in lexes:
            tmp['LexFound'] = lex
            K = dfextra.columns.max()+1
            K = 0 if np.isnan(K) else K
            dfextra[K] = tmp
            total += 1
        del tmp
dflat = dflat.T
dfextra = dfextra.T

dflat = pd.concat([dflat,dfextra])
print(dflat.shape)
print('save First')
dflat.to_csv(
    '../data/to-annotate/{}_all.csv'.format(COMMENT_FILES_FOLDER.split('/')[-1]),
    index=False)

100%|██████████| 493/493 [00:00<00:00, 768.13it/s](589, 11)
save First



In [55]:
mask = [url.startswith(fbpages) for url in dflat.PostURL]
sum(mask)

2

In [56]:
# dflat.iloc[:339].to_csv(
#     '../data/to-annotate/{}_339.csv'.format(dflat.postId.values[0].split('_')[0]),
#     index=False)

In [57]:
## Only take 10 per lex ## not needed since we need more sentences
# # max 10 per lexicon
# dfnew = []
# for lex in dflat.LexFound.unique():
#     dfnew.append(dflat.loc[dflat.LexFound==lex,:].iloc[:30])
# dfnew = pd.concat(dfnew).sort_values('LexFound', ascending=True)

In [58]:
# print(dfnew.shape)
# dfnew.head(2)

## Combine with Priority FB Page Data

In [59]:
dfwpg = dfwpg[['LexFound','postId','cId','nId','MsgUniSeg','Profile ID','PostURL','Date']]

In [60]:
dfwpg['date'] = [d.split()[0] for d in dfwpg.Date]
dfwpg['time'] = [d.split()[1] for d in dfwpg.Date]
dfwpg['ProfileUrl'] = ['{}{}'.format(URL_PREFIX, i.split()[-1]) for i in dfwpg['Profile ID']]
print(dfwpg.shape)
dfwpg.head(2)

(267, 11)


,LexFound,postId,cId,nId,MsgUniSeg,Profile ID,PostURL,Date,date,time,ProfileUrl
44854,NaN,20200430_0155,1,0,ပြည် သူ တွေ ငတ် သေ နေ ပြီ အစိုးရ ဘာ လုပ် နေ လဲ...,ID: 100008337798030,https://www.facebook.com/VoiceofMyanmarNews/po...,2020-05-05 07:23:33,2020-05-05,07:23:33,https://www.facebook.com/profile.php?id=100008...
44855,NaN,20200430_0155,1,1,SiThuAung အကုန် ကို ဗစ် ကူး တော့ မင်း တာဝန် ယူ...,ID: 100008619974774,https://www.facebook.com/VoiceofMyanmarNews/po...,2020-05-05 08:14:10,2020-05-05,08:14:10,https://www.facebook.com/profile.php?id=100008...


In [61]:
'../data/to-annotate/{}_final.csv'.format(COMMENT_FILES_FOLDER.split('/')[-1])

'../data/to-annotate/20200430_20200507_final.csv'

In [62]:
if True:
    dffinal = pd.concat([dflat, dfwpg])
    print(dffinal.shape)
    dffinal.drop_duplicates('MsgUniSeg', inplace=True)
    print(dffinal.shape)
    display(dffinal.head())
    # dffinal.iloc[:DesiredN,:].to_csv(
    #     '../data/to-annotate/{}_final.csv'.format(dffinal.postId.values[0].split('_')[0]),
    #     index=False)
else:
    dffinal = dflat

dffinal.iloc[:DesiredN,:].to_csv(
    '../data/to-annotate/{}_final.csv'.format(COMMENT_FILES_FOLDER.split('/')[-1]),
    index=False)

(856, 12)
(758, 12)


,LexFound,postId,cId,nId,MsgUniSeg,Profile ID,PostURL,CovFound,date,time,ProfileUrl,Date
0,သူခိုး,20200430_0200,11,0,ကျပ် ငွေ 200000( ၂ သိန်း ) ကို ဖုန်း လေး တစ် ခ...,ID: 100043931886875,https://www.facebook.com/yangonregiongovernmen...,covid,2020-05-06,19:03:54,https://www.facebook.com/profile.php?id=100043...,NaN
1,မြို့သား,20200430_0055,15,0,ကျွန်မ က မြ ဝ တီ သူမ ဟုတ် ပါ ဘူး ။ ရန် ကုန် က ...,ID: 100045639997751,https://www.facebook.com/1233832270078079/post...,မြို့သား,2020-05-03,04:11:45,https://www.facebook.com/profile.php?id=100045...,NaN
2,မြို့သား,20200430_0055,98,0,အဓိက က မြို့ ရဲ့ ဂုဏ်သိက္ခာ နစ်နာ တယ် ရောဂါကပ်...,ID: 100005686590684,https://www.facebook.com/1233832270078079/post...,မြို့သား~ရောဂါ,2020-05-03,07:28:36,https://www.facebook.com/profile.php?id=100005...,NaN
3,ဝူဟန်,20200430_0055,187,0,စား ရ လို့ မေး တာ ပေါ့ သူ့ အလုတ် ငွေ ဝင် တဲ့ န...,ID: 100036259579824,https://www.facebook.com/1233832270078079/post...,ဝူဟန်,2020-05-03,12:49:03,https://www.facebook.com/profile.php?id=100036...,NaN
4,အောက်တန်းစား,20200430_0112,39,0,ဘာ ပညာ တွေ လည်း အောက်တန်းစား ပညာ ဆေး ပညာ တွေ လ...,ID: 100028207507275,https://www.facebook.com/shwewikimyanmaryangon...,တရုတ်,2020-05-02,16:54:21,https://www.facebook.com/profile.php?id=100028...,NaN


In [34]:
dffinal.shape

(357, 12)